In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'
os.environ["CUDA_VISIBLE_DEVICES"]=" "

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib
matplotlib.rc("font",family='YouYuan')
%run Inception_Layer.ipynb

In [3]:
filepath=r"C:\Users\Administrator\Desktop\study\Datasets\Kaggle\archive\raw-img"
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(filepath, batch_size=18326,image_size = (227, 227), label_mode = "int", validation_split = .3, subset = "training", seed = 27)
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(filepath, batch_size=7853,image_size = (227, 227), label_mode = "int", validation_split = .3, subset = "validation", seed = 27)
for train_x,train_y in train_dataset:
    train_x=train_x/255.0
for validation_x,validation_y in validation_dataset:
    validation_x=validation_x/255.0
train_dataset=tf.data.Dataset.from_tensor_slices((train_x,(train_y,train_y,train_y)))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_x,(validation_y,validation_y,validation_y)))
"""train_dataset=tf.data.Dataset.from_tensor_slices((train_x,train_y)).batch(200)
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_x,validation_y)).batch(200)"""
train_dataset=train_dataset.shuffle(buffer_size=20000).batch(200)
validation_dataset=validation_dataset.shuffle(buffer_size=20000).batch(200)

Found 26179 files belonging to 10 classes.
Using 18326 files for training.
Found 26179 files belonging to 10 classes.
Using 7853 files for validation.


In [17]:
#完整版
inception_3a=Inception([16,24,32,4,8,8])
inception_3b=Inception([32,32,24,8,24,16])
inception_4a=Inception([48,24,24,4,12,16])
inception_4b=Inception([40,24,24,6,16,16])
inception_4c=Inception([32,32,32,6,16,16])
inception_4d=Inception([28,36,32,8,16,16])
inception_4e=Inception([32,40,80,8,32,32])
inception_5a=Inception([32,40,40,8,32,32])
inception_5b=Inception([48,24,48,12,32,32])
regular=tf.keras.regularizers.l2(0.01)
input=keras.Input(shape=(227,227,3))
C1=layers.Conv2D(16,kernel_size=(7,7),strides=2,padding="same",activation="relu",kernel_regularizer=regular)(input)
P2=layers.MaxPool2D(pool_size=3,strides=2,padding="same")(C1)
B3=layers.BatchNormalization()(P2)
C4=layers.Conv2D(16,kernel_size=(1,1),padding="valid",activation="relu",kernel_regularizer=regular)(B3)
C5=layers.Conv2D(48,kernel_size=(3,3),padding="same",activation="relu",kernel_regularizer=regular)(C4)
B6=layers.BatchNormalization()(C5)
P7=layers.MaxPool2D(pool_size=3,strides=2,padding="same")(B6)
I8=inception_3a(P7)
I9=inception_3b(I8)
P10=layers.MaxPool2D(pool_size=3,strides=2,padding="same")(I9)
I11=inception_4a(P10)

S1_P12=layers.AveragePooling2D(pool_size=5,strides=3,padding="valid")(I11)
S1_C13=layers.Conv2D(64,kernel_size=(1,1),activation="relu",padding="same",kernel_regularizer=regular)(S1_P12)
S1_F14=layers.Flatten()(S1_C13)
S1_F15=layers.Dense(64,activation="relu",kernel_regularizer=regular)(S1_F14)
S1_D16=layers.Dropout(0.5)(S1_F15)
S1_F17=layers.Dense(128,activation="relu",kernel_regularizer=regular)(S1_D16)
S1_D18=layers.Dropout(0.5)(S1_F17)
S1_output=layers.Dense(10,activation="softmax",name="S1_output")(S1_D18)

I12=inception_4b(I11)
I13=inception_4c(I12)
I14=inception_4d(I13)

S2_P15=layers.AveragePooling2D(pool_size=5,strides=3,padding="valid")(I14)
S2_C16=layers.Conv2D(64,kernel_size=(1,1),activation="relu",padding="same",kernel_regularizer=regular)(S2_P15)
S2_B17=layers.BatchNormalization()(S2_C16)
S2_F18=layers.Flatten()(S2_B17)
S2_F19=layers.Dense(64,activation="relu",kernel_regularizer=regular)(S2_F18)
S2_D20=layers.Dropout(0.5)(S2_F19)
S2_F21=layers.Dense(128,activation="relu",kernel_regularizer=regular)(S2_D20)
S2_D22=layers.Dropout(0.5)(S2_F21)
S2_output=layers.Dense(10,activation="softmax",name="S2_output")(S2_D22)

I15=inception_4e(I14)
P16=layers.MaxPool2D(pool_size=3,strides=2,padding="same")(I15)
I17=inception_5a(P16)
I18=inception_5b(I17)
P19=layers.AveragePooling2D(pool_size=7,strides=1,padding="valid")(I18)
F20=layers.Flatten()(P19)
F21=layers.Dense(128,activation="relu",kernel_regularizer=regular)(F20)
D22=layers.Dropout(0.5)(F21)
M_output=layers.Dense(10,activation="softmax",name="M_output")(D22)
model=keras.Model(inputs=input,outputs=[S1_output,M_output,S2_output])
optimizer = optimizers.Adam(learning_rate=0.001,weight_decay=0.001)
model.compile(optimizer=optimizer,
              loss={
                  'M_output':'sparse_categorical_crossentropy',
                  'S1_output':'sparse_categorical_crossentropy',
                  'S2_output':'sparse_categorical_crossentropy'
             },
              loss_weights={
                  'M_output':1.0,
                  'S1_output':0.2,
                  'S2_output':0.3,
             },
              metrics=['accuracy','accuracy','accuracy'],)
model_checkpoint=ModelCheckpoint("inception_v1.keras",save_best_only=True)
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [ ]:
history=model.fit(train_dataset,epochs=50,
          validation_data=validation_dataset,
          shuffle=True,callbacks=[model_checkpoint])

Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - M_output_accuracy: 0.2058 - S1_output_accuracy: 0.1866 - S2_output_accuracy: 0.1927 - loss: 8.7166 - val_M_output_accuracy: 0.2451 - val_S1_output_accuracy: 0.1747 - val_S2_output_accuracy: 0.1747 - val_loss: 4.6709
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 233s 1s/step - M_output_accuracy: 0.3528 - S1_output_accuracy: 0.3153 - S2_output_accuracy: 0.3861 - loss: 3.7906 - val_M_output_accuracy: 0.1747 - val_S1_output_accuracy: 0.1747 - val_S2_output_accuracy: 0.1746 - val_loss: 3.8943
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 224s 1s/step - M_output_accuracy: 0.4343 - S1_output_accuracy: 0.3906 - S2_output_accuracy: 0.4622 - loss: 2.9114 - val_M_output_accuracy: 0.2458 - val_S1_output_accuracy: 0.2162 - val_S2_output_accuracy: 0.1867 - val_loss: 3.5949
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 238s 2s/step - M_output_accuracy: 0.4780 - S1_output_accuracy: 0.4245 - S2_output_accuracy: 0.5063 - loss: 2.5832 - val_M_output_accuracy: 0.2227 - val_S1_